## Simple guide on getting sequences for lstm model

In [1]:
import edec.afterprocessing as ap
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.functional import normalize
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### Create tin lists and data sequence for train

In [2]:
# Pick out which tin you want in a split.
train_tin_fail, train_tin_healthy = ap.get_subset_data(dataset="train", n_healthy=44, n_failed=44, separate=True)

#features_single=["Soc_Dev", "Volt_Dev", "Age"]
features_single=["Soc_Dev"]

# Should make this to one function call, but right now needs to separate healthy and faulty tin as input
train_fail = ap.create_sequences_lstm(
    train_tin_fail, faulty=True, features_single=None, features_nest=["Sorted_SOC"], n_history=2)
train_healthy = ap.create_sequences_lstm(
    train_tin_healthy, faulty=True, features_single=None, features_nest=["Sorted_SOC"], n_history=2)

# Add healthy and faulty samples
train_data = train_fail+train_healthy

### Create tin lists and data sequence for vaidation

In [14]:
# Pick out which tin you want in a split.
val_tin_fail, val_tin_healthy = ap.get_subset_data(dataset="val", n_healthy=44, n_failed=44, separate=True)

# Should make this to one function call, but right now needs to separate healthy and faulty tin as input
val_fail = ap.create_sequences_lstm(
    val_tin_fail, faulty=True, features_single=None, features_nest=["Sorted_SOC"], n_history=2)
val_healthy = ap.create_sequences_lstm(
    val_tin_healthy, faulty=False, features_single=None, features_nest=["Sorted_SOC"], n_history=2)

# Add healthy and faulty samples
val_data = val_fail+val_healthy

In [16]:
val_data[0][0].shape

torch.Size([2, 270])

## Create dataset

In [4]:
batch_size = 10
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True,drop_last=True)
test_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=True,drop_last=True)

In [9]:
for b, (X_test,y_test) in enumerate(train_loader):
    break
X_test.shape

torch.Size([10, 2, 270])

In [11]:
for b, (X_test,y_test) in enumerate(test_loader):
    break
X_test.shape

RuntimeError: stack expects each tensor to be equal size, but got [2, 270] at entry 0 and [2, 216] at entry 5

In [102]:
model(X_train.view(batch_size, -1))

RuntimeError: expected scalar type Float but found Double

In [34]:
 torch.FloatTensor

## Create network

In [89]:
train_data[0][0].shape

torch.Size([5, 109])

In [90]:
sub_samples, features = list(train_data[0][0].shape)

In [91]:
class ANNMultilayerperceptron(nn.Module):

    def __init__(self, input_size=(sub_samples*features),output_size=2, layers=[220,84]):  # 120, 84
        super().__init__()

        self.fc1 = nn.Linear(input_size, layers[0])
        self.fc2 = nn.Linear(layers[0], layers[1])
        self.fc2b = nn.Linear(layers[1], 500)
        self.fc2c = nn.Linear(500, layers[1])
        self.fc2d = nn.Linear(layers[1], layers[1])
        self.fc3 = nn.Linear(layers[1], output_size)

    def forward(self,X):
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = F.relu(self.fc2b(X))
        X = F.relu(self.fc2c(X))
        X = F.relu(self.fc2d(X))
        X = self.fc3(X)

        return F.log_softmax(X,dim=1) # PGA multiclass classification
        #return X

In [92]:
model = ANNMultilayerperceptron()
model

ANNMultilayerperceptron(
  (fc1): Linear(in_features=545, out_features=220, bias=True)
  (fc2): Linear(in_features=220, out_features=84, bias=True)
  (fc2b): Linear(in_features=84, out_features=500, bias=True)
  (fc2c): Linear(in_features=500, out_features=84, bias=True)
  (fc2d): Linear(in_features=84, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)

In [93]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
#optimizer = torch.optim.Adadelta(model.parameters(), lr=0.001)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [94]:
X_train[0][0]

tensor([1.3100e+00, 1.8321e-03, 1.4400e+05, 4.0976e+00, 4.0912e+00, 4.0906e+00,
        4.0975e+00, 4.0973e+00, 4.0916e+00, 4.0912e+00, 4.0978e+00, 4.0905e+00,
        4.0921e+00, 4.0908e+00, 4.0953e+00, 4.1004e+00, 4.0992e+00, 4.0986e+00,
        4.1007e+00, 4.1008e+00, 4.0992e+00, 4.0988e+00, 4.1007e+00, 4.1012e+00,
        4.0999e+00, 4.0998e+00, 4.1011e+00, 4.0985e+00, 4.0939e+00, 4.0952e+00,
        4.0937e+00, 4.0991e+00, 4.0922e+00, 4.0939e+00, 4.0997e+00, 4.0921e+00,
        4.0931e+00, 4.0932e+00, 4.0932e+00, 4.0911e+00, 4.0927e+00, 4.0922e+00,
        4.0939e+00, 4.0907e+00, 4.0936e+00, 4.0942e+00, 4.0981e+00, 4.0911e+00,
        4.0929e+00, 4.0934e+00, 4.0936e+00, 4.0946e+00, 4.0899e+00, 4.0900e+00,
        4.0899e+00, 4.0960e+00, 4.0898e+00, 4.0899e+00, 4.0913e+00, 4.0918e+00,
        4.0922e+00, 4.0893e+00, 4.0941e+00, 4.0983e+00, 4.0941e+00, 4.0944e+00,
        4.0932e+00, 4.0916e+00, 4.0946e+00, 4.0943e+00, 4.0972e+00, 4.0914e+00,
        4.0941e+00, 4.0940e+00, 4.0940e+

In [95]:
for b, (X_train, y_train) in enumerate(train_loader):
    break

In [96]:
model(X_train[0][0].view(batch_size, -1))

RuntimeError: shape '[10, -1]' is invalid for input of size 109

In [99]:
X_train[0][0].view(10,-1)

RuntimeError: shape '[10, -1]' is invalid for input of size 109